## Preprocessing the data

In [1]:
import numpy as np
from sklearn import preprocessing #sklearn for standardization of inputs

#Standardization improves the quality of the algorithm

In [2]:
raw_csv_data = np.loadtxt('Audiobooks_data.csv', delimiter = ',')

In [4]:
unscaled_inputs_all = raw_csv_data[:,1:-1] # first column is not needed, last column is targets
targets_all = raw_csv_data[:,-1]

## Balance the data

In [5]:
num_one_targets = int(np.sum(targets_all)) # Only those targets will contribute to the sum which are 1s.

zero_targets_counter = 0 # To count the number of zeros over the targets
indices_to_remove = [] # To remove the unwanted zeros for balancing

for i in range(targets_all.shape[0]):
    if targets_all[i] == 0: # check for zero
        zero_targets_counter += 1 # increment the counter
        if zero_targets_counter > num_one_targets: # note the index once the counter is greater than the number of 1s
            indices_to_remove.append(i)
            
unscaled_inputs_equal_priors = np.delete(unscaled_inputs_all, indices_to_remove, axis = 0) # removing the indices in inputs

targets_equal_priors = np.delete(targets_all, indices_to_remove, axis = 0) # removing indices in targets

## Standardize the inputs

In [8]:
scaled_inputs = preprocessing.scale(unscaled_inputs_equal_priors) # Standardizing the inputs using sklearn
scaled_inputs

array([[ 0.21053387, -0.18888517,  1.97823887, ...,  4.80955413,
        11.83828419,  0.09415043],
       [ 1.27894497,  0.41646744, -0.39082475, ..., -0.41569922,
        -0.20183481, -0.80255852],
       [ 1.27894497,  0.41646744, -0.39082475, ..., -0.41569922,
        -0.20183481,  2.979214  ],
       ...,
       [ 1.27894497,  0.41646744, -0.39082475, ..., -0.41569922,
        -0.20183481, -0.7440775 ],
       [ 0.31737498,  1.7482432 ,  0.04679395, ..., -0.41569922,
        -0.20183481, -0.80255852],
       [ 0.31737498,  1.7482432 , -0.39082475, ..., -0.41569922,
        -0.20183481, -0.80255852]])

## Shuffle the data

In [10]:
shuffled_indices = np.arange(scaled_inputs.shape[0]) # Take the indices of the inputs in the first column and store them
np.random.shuffle(shuffled_indices) # Shuffle these indices

shuffled_inputs = scaled_inputs[shuffled_indices] # Picking and storing the inputs corresponding to shuffled indices
shuffled_targets = targets_equal_priors[shuffled_indices] # Picking and storing targets corresponding to shuffled indices

## Split into train, validation and test data

In [14]:
samples_count = shuffled_inputs.shape[0] # number of samples

# Use of 80-10-10 split here

train_samples_count = int(0.8*samples_count)
validation_samples_count = int(0.1*samples_count)
test_samples_count = samples_count - train_samples_count - validation_samples_count

train_inputs = shuffled_inputs[:train_samples_count] # Extracting training data from inputs
train_targets = shuffled_targets[:train_samples_count] # Likewise for targets

validation_inputs = shuffled_inputs[train_samples_count:validation_samples_count+train_samples_count] # from train to train+val
validation_targets = shuffled_targets[train_samples_count:validation_samples_count+train_samples_count]

test_inputs = shuffled_inputs[train_samples_count+validation_samples_count:] # Everything left is the test
test_targets = shuffled_targets[train_samples_count+validation_samples_count:]



## Save the three datasets in npz arrays

In [15]:
np.savez('Audiobooks_data_train', inputs = train_inputs, targets = train_targets)
np.savez('Audiobooks_data_validation', inputs = validation_inputs, targets = validation_targets)
np.savez('Audiobooks_data_test', inputs = test_inputs, targets = test_targets)